<pre>
   _____                            _                __      ___     _                  _                        _ 
  / ____|                          | |               \ \    / (_)   (_)                | |                      | |
 | |     ___  _ __ ___  _ __  _   _| |_ ___ _ __      \ \  / / _ ___ _  ___  _ __      | |__   __ _ ___  ___  __| |
 | |    / _ \| '_ ` _ \| '_ \| | | | __/ _ \ '__|      \ \/ / | / __| |/ _ \| '_ \     | '_ \ / _` / __|/ _ \/ _` |
 | |___| (_) | | | | | | |_) | |_| | ||  __/ |          \  /  | \__ \ | (_) | | | |    | |_) | (_| \__ \  __/ (_| |
  \_____\___/|_| |_| |_| .__/ \__,_|\__\___|_|           \/   |_|___/_|\___/|_| |_|    |_.__/ \__,_|___/\___|\__,_|
  _______              | | __  __                                                    _                             
 |__   __|             |_||  \/  |                                                  | |                            
    | |_ __ ___  ___      | \  / | ___  __ _ ___ _   _ _ __ ___ _ __ ___   ___ _ __ | |_                           
    | | '__/ _ \/ _ \     | |\/| |/ _ \/ _` / __| | | | '__/ _ \ '_ ` _ \ / _ \ '_ \| __|                          
    | | | |  __/  __/     | |  | |  __/ (_| \__ \ |_| | | |  __/ | | | | |  __/ | | | |_                           
    |_|_|  \___|\___|     |_|  |_|\___|\__,_|___/\__,_|_|  \___|_| |_| |_|\___|_| |_|\__|                          
                                                                                                                   
                                                                                                                   

# Global Settings

In [2]:
# Assets folder
ASSETS = "./assets"


# Video's
CALIBRATION_VIDEO = f"{ASSETS}/original/calibration.MP4"
EASTBOUND_VIDEO   = f"{ASSETS}/original/eastbound_20240319.MP4"
WESTBOUND_VIDEO   = f"{ASSETS}/original/westbound_20240319.MP4"

# Preliminary

### imports

In [3]:
# imports

import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt



# When running code in JupyterHub / Google Colab, Opencv might not be
# installed. This piece of code installs it when it is not yet available.
try:
    import cv2
    print("Succesfully imported OpenCV")
except ImportError:
    print("OpenCV is not installed, installing now")

    !pip install opencv-python


# installing detectron2
!pip install 'git+https://github.com/facebookresearch/detectron2.git'


from __future__ import  absolute_import

# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

import torch
import json
import gc

# import detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.video_visualizer import VideoVisualizer

Succesfully imported OpenCV


# Image Processing
## Frame Extraction

In this section, we extract individual frames from the video. This process involves loading the video file, iterating through each $k$ frames, and saving these frames as separate image files. Each extracted frame is named to include its frame number, ensuring a clear and organized sequence. Extracting frames allows for detailed analysis and processing of each moment captured in the video.

In [5]:
# frame extraction from video (keeping the original frame number)
# this could help us in the case where we cannot detect trees in certain frames, then we can use the original frame number to extract more frames from the original video in this time area

def extract_frames(video_path, capture_every_frame=5, output_folder=F"{ASSETS}/extracted"):
    # Adjust the output folder to include how many frames were skipped
    output_folder += f"_{capture_every_frame:02d}"
    
    # Extract video name from path
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    
    # Create the directory if it doesn't exist
    os.makedirs(f"{output_folder}/{video_name}", exist_ok=True)

    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Get total number of frames
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in {video_name}: {total_frames}")

    # Initialize the frame counter
    current_frame = 0

    # Process frames
    while current_frame < total_frames:
        video.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
        success, image = video.read()

        # Check if the frame was successfully read
        # sometimes this fails, corrupt frames in vide? idk
        # using frames:05d (file_name_frame_00001.png) for easy sorting later on
        if success:
            # Save the frame with the actual frame number in the file name
            cv2.imwrite(f"{output_folder}/{video_name}/{video_name}_{current_frame:05d}.png", image)
            print(f"{output_folder}/{video_name}/{video_name}_{current_frame:05d}.png")
            # Skip to the next frame based on the specified interval
            current_frame += capture_every_frame
        else:
            # Output an error message if the frame failed to extract
            print(f"Frame {current_frame} failed to extract")
            # Try the next frame instead of skipping the interval because we couldn't read the current frame
            current_frame += 1


    # Release the video capture object
    video.release()


In [6]:
extract_frames(CALIBRATION_VIDEO, capture_every_frame=30)

Total frames in calibration: 2995
./assets/extracted_30/calibration/calibration_00000.png
./assets/extracted_30/calibration/calibration_00030.png
./assets/extracted_30/calibration/calibration_00060.png
./assets/extracted_30/calibration/calibration_00090.png
./assets/extracted_30/calibration/calibration_00120.png
./assets/extracted_30/calibration/calibration_00150.png
./assets/extracted_30/calibration/calibration_00180.png
./assets/extracted_30/calibration/calibration_00210.png
./assets/extracted_30/calibration/calibration_00240.png
./assets/extracted_30/calibration/calibration_00270.png
./assets/extracted_30/calibration/calibration_00300.png
./assets/extracted_30/calibration/calibration_00330.png
./assets/extracted_30/calibration/calibration_00360.png
./assets/extracted_30/calibration/calibration_00390.png
./assets/extracted_30/calibration/calibration_00420.png
./assets/extracted_30/calibration/calibration_00450.png
./assets/extracted_30/calibration/calibration_00480.png
./assets/extra

In [7]:
extract_frames(EASTBOUND_VIDEO)

Total frames in eastbound_20240319: 18786
./assets/extracted_05/eastbound_20240319/eastbound_20240319_00000.png
./assets/extracted_05/eastbound_20240319/eastbound_20240319_00005.png
./assets/extracted_05/eastbound_20240319/eastbound_20240319_00010.png
./assets/extracted_05/eastbound_20240319/eastbound_20240319_00015.png
./assets/extracted_05/eastbound_20240319/eastbound_20240319_00020.png
./assets/extracted_05/eastbound_20240319/eastbound_20240319_00025.png
./assets/extracted_05/eastbound_20240319/eastbound_20240319_00030.png
./assets/extracted_05/eastbound_20240319/eastbound_20240319_00035.png
./assets/extracted_05/eastbound_20240319/eastbound_20240319_00040.png
./assets/extracted_05/eastbound_20240319/eastbound_20240319_00045.png
Frame 50 failed to extract
Frame 51 failed to extract
Frame 52 failed to extract
Frame 53 failed to extract
Frame 54 failed to extract
Frame 55 failed to extract
Frame 56 failed to extract
Frame 57 failed to extract
Frame 58 failed to extract
Frame 59 failed

In [8]:
extract_frames(WESTBOUND_VIDEO)

Total frames in westbound_20240319: 12371
./assets/extracted_05/westbound_20240319/westbound_20240319_00000.png
./assets/extracted_05/westbound_20240319/westbound_20240319_00005.png
./assets/extracted_05/westbound_20240319/westbound_20240319_00010.png
./assets/extracted_05/westbound_20240319/westbound_20240319_00015.png
./assets/extracted_05/westbound_20240319/westbound_20240319_00020.png
./assets/extracted_05/westbound_20240319/westbound_20240319_00025.png
./assets/extracted_05/westbound_20240319/westbound_20240319_00030.png
./assets/extracted_05/westbound_20240319/westbound_20240319_00035.png
./assets/extracted_05/westbound_20240319/westbound_20240319_00040.png
./assets/extracted_05/westbound_20240319/westbound_20240319_00045.png
Frame 50 failed to extract
Frame 51 failed to extract
Frame 52 failed to extract
Frame 53 failed to extract
Frame 54 failed to extract
Frame 55 failed to extract
Frame 56 failed to extract
Frame 57 failed to extract
Frame 58 failed to extract
Frame 59 failed

## Calibration

In this section, we calculate the camera calibration matrices using Zhang's method, which is a widely used technique for camera calibration in computer vision. This method leverages multiple images of a known calibration pattern, in this case a chessboard, to estimate the camera's intrinsic and extrinsic parameters. By utilizing Zhang's method, this calibration process enables accurate determination of the camera's parameters, which are essential for correcting lens distortion and improving the accuracy of subsequent image processing tasks.

In [9]:
def calibrate_camera(calibration_images):
    objp = np.zeros((6*8, 3), np.float32)
    objp[:, :2] = np.mgrid[0:8, 0:6].T.reshape(-1, 2)

    objpoints = []
    imgpoints = []


    for img_path in calibration_images:
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        ret, corners = cv2.findChessboardCorners(gray, (8, 6), None)

        if ret:
            objpoints.append(objp)
            imgpoints.append(corners)


    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    return ret, mtx, dist, rvecs, tvecs

In [11]:
calibration_images = glob(f"{ASSETS}/extracted_30/calibration/*.png")

ret, mtx, dist, rvecs, tvecs = calibrate_camera(calibration_images)

# save data for later use
np.save(f"{ASSETS}/calibration_data/intrinsic_parameters.npy", mtx)
np.save(f"{ASSETS}/calibration_data/distortion_coefficients.npy", dist)
np.save(f"{ASSETS}/calibration_data/rotation_vectors.npy", rvecs)
np.save(f"{ASSETS}/calibration_data/translation_vectors.npy", tvecs)

FileNotFoundError: [Errno 2] No such file or directory: './assets/calibration_data/intrinsic_parameters.npy'

## Undistortion
After calibrating the camera and obtaining the necessary calibration matrices, the next step is to undistort the images. Lens distortion, which manifests as warping or curving of straight lines in images, is a common issue in photography, especially with wide-angle lenses. Using the calibration data derived from Zhang's method, we can correct this radial distortion and produce geometrically accurate images.

In [8]:
def undistort_images(images, output_loc=f"{ASSETS}/undistorted"):
    os.makedirs(f"{output_loc}", exist_ok=True)

    for img_path in images:
        img = cv2.imread(img_path)
        frame_name = os.path.splitext(os.path.basename(img_path))[0]
        
        undistorted_image = cv2.undistort(img, mtx, dist, None, mtx)
        cv2.imwrite(f"./{output_loc}/{frame_name}.png", undistorted_image)

In [9]:
# For now only testing with 200 images (they have easy to detect trees on the left and hard to detect trees on the right)
# Run the code below to generate the undistorted images
# or you can also download the undistorted images from here: https://ugentbe-my.sharepoint.com/:f:/g/personal/thomas_dirven_ugent_be/Eu76kcdu3HJJsbQwTMyyDZwBH_quW09UIMCKP4AFmAiJtA?e=wROqbx
# Download undistored_05.zip (1.12 GB) in shared folder

# westbound_images   = glob('./assets/extracted_05/westbound_20240319/*.png')
eastbound_images   = glob('./assets/extracted_05/eastbound_20240319/*.png')
eastbound_images.sort()
# interesting images at 7830
# divide by 5 = 1566
# then take 200 images for testing
eastbound_images = eastbound_images[1560:1760]
# print(eastbound_images)

# undistort_images(westbound_images, output_loc="./assets/undistorted/westbound")
undistort_images(eastbound_images, output_loc="./assets/undistorted_05/eastbound")

# Tree Detection - TODO Clean Up

## PercepTree



In [ ]:
# Code from PercepTreeV1 use as test demo

# local paths to model and image
# model_name = 'X-101_RGB_60k.pth'
# model_name = 'R-50_RGB_60k.pth'
model_name = 'ResNext-101_fold_01.pth'
image_path = './output/image_00000_RGB.png'

if __name__ == "__main__":
    torch.cuda.is_available()
    logger = setup_logger(name=__name__)
    
    # All configurables are listed in /repos/detectron2/detectron2/config/defaults.py        
    cfg = get_cfg()
    cfg.INPUT.MASK_FORMAT = "bitmask"
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml"))
    # cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml"))
    # cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
    cfg.DATASETS.TRAIN = ()
    cfg.DATASETS.TEST = ()
    cfg.DATALOADER.NUM_WORKERS = 8
    cfg.SOLVER.IMS_PER_BATCH = 8
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256   # faster (default: 512)
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (tree)
    cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 1  
    cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 5
    cfg.MODEL.MASK_ON = True
    
    cfg.OUTPUT_DIR = './output'
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, model_name)
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
    # cfg.INPUT.MIN_SIZE_TEST = 0  # no resize at test time
    
    # set detector
    predictor_synth = DefaultPredictor(cfg)    
    
    # set metadata
    tree_metadata = MetadataCatalog.get("my_tree_dataset").set(thing_classes=["Tree"], keypoint_names=["kpCP", "kpL", "kpR", "AX1", "AX2"])
    
    # inference
    im = cv2.imread(image_path)
    outputs_pred = predictor_synth(im)
    v_synth = Visualizer(im[:, :, ::-1],
                    metadata=tree_metadata, 
                    scale=1,
    )
    out_synth = v_synth.draw_instance_predictions(outputs_pred["instances"].to("cpu"))

    # Assuming out_synth.get_image() returns the image
    image = out_synth.get_image()[:, :, ::-1]  # Assuming the image is in BGR format, converting it to RGB
    
    plt.figure(figsize=(20, 10))
    plt.imshow(image)
    # plt.axis('off')  # Turn off axis
    plt.show()

    # Original code from demo, but this give errors in online notebooks, using above matplotlib instead
    # cv2.imshow('predictions', out_synth.get_image()[:, :, ::-1])
    # k = cv2.waitKey(0)
    
    # cv2.destroyAllWindows()  
        

In [ ]:
predictions = outputs_pred["instances"].to("cpu")

In [ ]:
num_instances = len(predictions.pred_boxes)
image_height = predictions.image_size[0]
image_width = predictions.image_size[1]

print(num_instances)
print(image_height)
print(image_width)

In [ ]:
predictions.get("pred_boxes").tensor.tolist()

In [ ]:
# Just showing the first two sets of 5 keypoints

predictions.get("pred_keypoints").tolist()[:2]

In [ ]:
# Some global vars (temp location, will be moved into function call, def process_list_of_images(display_image, some_other_vars,...): )
display_image = True

# local paths to model and image
# model_name = 'X-101_RGB_60k.pth'
model_name = 'ResNext-101_fold_01.pth'
base_path = './assets/undistorted_05/eastbound'
image_dir_pattern = base_path + '/*.png'
output_dir = './assets/annotated_05/eastbound/'

# Ensure that the output directory exists, create it if necessary
os.makedirs(output_dir, exist_ok=True)

image_paths = glob(image_dir_pattern)
image_paths.sort()
print("Images to process:", len(image_paths))

# def process_list_of_images():
torch.cuda.is_available()
logger = setup_logger(name=__name__)

# All configurables are listed in /repos/detectron2/detectron2/config/defaults.py        
cfg = get_cfg()
cfg.INPUT.MASK_FORMAT = "bitmask"
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml"))
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml"))
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ()
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 8
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256   # faster (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (tree)
cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 1  
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 5
cfg.MODEL.MASK_ON = True

cfg.OUTPUT_DIR = './output'
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, model_name)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
# cfg.INPUT.MIN_SIZE_TEST = 0  # no resize at test time

# set detector
predictor_synth = DefaultPredictor(cfg)    

# set metadata
tree_metadata = MetadataCatalog.get("my_tree_dataset").set(thing_classes=["Tree"], keypoint_names=["kpCP", "kpL", "kpR", "AX1", "AX2"])

for image_path in image_paths:    
    file_name = image_path.split("/")[-1]
    output_path = output_dir + file_name
    # inference
    im = cv2.imread(image_path)
    outputs_pred = predictor_synth(im)
    v_synth = Visualizer(im[:, :, ::-1],
                    metadata=tree_metadata, 
                    scale=1,
    )
    predictions = outputs_pred["instances"].to("cpu")
    out_synth = v_synth.draw_instance_predictions(predictions)

    # Assuming out_synth.get_image() returns the image
    image = out_synth.get_image()[:, :, ::-1]  # Assuming the image is in BGR format, converting it to RGB
    
    # Save the image
    cv2.imwrite(output_path, image)
    print("Succesfully wrote image to:", output_path)
    
    # Convert the tensors to lists
    pred_boxes_list = predictions.get("pred_boxes").tensor.tolist()
    scores_list = predictions.get("scores").tolist()
    pred_keypoints_list = predictions.get("pred_keypoints").tolist()
    
    # .tolist() on a tensor is extremely slow, so saved it as npy instead (you can test this in a seperate cell and see how slow it is)
    # pred_masks_list = predictions.get("pred_masks_list").tolist()
    # pred_keypoints_heatmaps_list = predictions.get("pred_keypoint_heatmaps").tolist()
    pred_mask_numpy = predictions.get("pred_masks").numpy()
    pred_keypoint_heatmaps_numpy = predictions.get("pred_keypoint_heatmaps").numpy()

    # File base name
    file_base_name = file_name.split(".png")[0]
    # File names are kept in json as reference to numpy array file
    pred_masks_file_name = file_base_name + '_pred_mask.npy'
    pred_keypoint_heatmaps_file_name = file_base_name + '_pred_keypoints_heatmaps.npy'
    # Create full path that is used for saving the .npy files
    pred_mask_numpy_path = output_dir + pred_masks_file_name
    pred_keypoint_heatmaps_numpy_path = output_dir + pred_keypoint_heatmaps_file_name
    # Save the .npy arrays
    print("Saving:", pred_mask_numpy_path)
    np.save(pred_mask_numpy_path, pred_mask_numpy)
    print("Saving:", pred_keypoint_heatmaps_numpy_path)
    np.save(pred_keypoint_heatmaps_numpy_path, pred_keypoint_heatmaps_numpy)
    
    # Prepare a dictionary for JSON serialization
    json_data = {
        "pred_boxes": pred_boxes_list,
        "scores": scores_list,
        "pred_keypoints": pred_keypoints_list,
        "pred_masks": pred_masks_file_name,
        "pred_keypoint_heatmaps": pred_keypoint_heatmaps_file_name,
    }
    
    # Save to a JSON file    
    file_base_name = file_name.split(".png")[0]
    json_path = output_dir + file_base_name + '.json'
    with open(json_path, 'w') as json_file:
        json.dump(json_data, json_file, indent=4)
    
    print(f"Data has been saved to {json_path}")    

    if display_image:        
        plt.figure(figsize=(20, 10))
        plt.imshow(out_synth.get_image())
        # plt.axis('off')  # Turn off axis
        plt.show()

    collected_garbage = gc.collect()
    print("Removed garbage:", collected_garbage)
    

In [ ]:
# UNUSED



# # Code used for video data - We are currently not using this (and are not planning to use it, I guess, individual frames might give better results)

# #  model and video variables
# model_name = 'R-50_RGB_60k.pth'
# # model_name = 'X-101_RGB_60k.pth'
# video_path = './output/forest_walk_1min.mp4'

# if __name__ == "__main__":
#     torch.cuda.is_available()
#     logger = setup_logger(name=__name__)
    
#     # All configurables are listed in /repos/detectron2/detectron2/config/defaults.py        
#     cfg = get_cfg()
#     cfg.INPUT.MASK_FORMAT = "bitmask"
#     # cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml"))
#     # cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml"))
#     cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
#     cfg.DATASETS.TRAIN = ()
#     cfg.DATASETS.TEST = ()
#     cfg.DATALOADER.NUM_WORKERS = 8
#     cfg.SOLVER.IMS_PER_BATCH = 8
#     cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256   # faster (default: 512)
#     cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (tree)
#     cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 1  
#     cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 5
#     cfg.MODEL.MASK_ON = True
    
#     cfg.OUTPUT_DIR = './output' 
#     cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, model_name)
#     cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
#     # cfg.INPUT.MIN_SIZE_TEST = 0  # no resize at test time
    
#     # set detector
#     predictor_synth = DefaultPredictor(cfg)    
    
#     # set metadata
#     tree_metadata = MetadataCatalog.get("my_tree_dataset").set(thing_classes=["Tree"], keypoint_names=["kpCP", "kpL", "kpR", "AX1", "AX2"])
            
#     # Get one video frame 
#     vcap = cv2.VideoCapture(video_path)
    
#     # get vcap property 
#     w = int(vcap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     h = int(vcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#     fps = int(vcap.get(cv2.CAP_PROP_FPS))
#     n_frames = int(vcap.get(cv2.CAP_PROP_FRAME_COUNT))
    
#     # VIDEO recorder
#     # Grab the stats from image1 to use for the resultant video
#     # fourcc = cv2.VideoWriter_fourcc(*'mp4v')   
#     # video = cv2.VideoWriter("pred_and_track_00.mp4",fourcc, 5, (w, h))  
    
#     # Check if camera opened successfully
#     if (vcap.isOpened()== False):
#         print("Error opening video stream or file")
       
#     vid_vis = VideoVisualizer(metadata=tree_metadata)
                                
#     nframes = 0
#     while(vcap.isOpened() ):
#         ret, frame = vcap.read()
#         # if frame is read correctly ret is True
#         if not ret:
#             print("Can't receive frame (stream end?). Exiting ...")
#             break
#         y = 000
#         # h = 800
#         x = 000
#         # w = 800
#         crop_frame = frame[y:y+h, x:x+w]
#         # cv2.imshow('frame', crop_frame)
#         if cv2.waitKey(1) == ord('q'):
#                 break
        
#         # 5 fps
#         if nframes % 12 == 0:
#             outputs_pred = predictor_synth(crop_frame)s
#             # v_synth = Visualizer(crop_frame[:, :, ::-1],
#             #                     metadata=tree_metadata, 
#             #                     scale=1, 
#             #                     instance_mode =  ColorMode.IMAGE     # remove color from image, better see instances  
#             #     )
#             out = vid_vis.draw_instance_predictions(crop_frame, outputs_pred["instances"].to("cpu"))
                
#             vid_frame = out.get_image()
#             # video.write(vid_frame)
#             # cv2.imshow('frame', vid_frame)

#             # Assuming out_synth.get_image() returns the image
#             image = out_synth.get_image()[:, :, ::-1]  # Assuming the image is in BGR format, converting it to RGB
            
#             plt.imshow(vid_frame)
#             plt.axis('off')  # Turn off axis
#             plt.show()
            
#         nframes += 1
    
#     # video.release()
#     vcap.release()
#     # cv2.destroyAllWindows()

# Tree Triangulation (Colmap part goes here)

In [ ]:
# colmap



# Tree Mapping (notebook Robbe, Wout)

# Tree Measurement

In [6]:
# work of Thomas (depth images)